**OBLIGATORIO FINAL G448 - Data Wrangling**
<br>**INTEGRANTES:**
<br>
+ Theo Pereira, 5703834-3, theopereira2001@gmail.com
+ Carlos Píriz, 4298784-4, piriz.carlos@gmail.com
+ Gabriel	Better,3138871-2, gabrielbetter@gmail.com

In [440]:
### Importacion de librerias ###

# Sistema
import os

# Tratamiento de datos
import pandas as pd
import numpy as np
import datetime as dt
import math

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import re

# Visualizacion 
import seaborn as sns
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter
from babel.numbers import format_currency
import matplotlib.pyplot as plt
%matplotlib inline

**CONSIGNAS**
<br>
El objetivo general es realizar un análisis exploratorio de los anuncios inmobiliarios de
algunas provincias de la Patagonia Argentina reportados por el portal Airbnb.
<br>
Los datos están conformados por cuatro archivos csv:
+ Neuquen.csv
+ Rio_Negro.csv
+ Chubut.csv
+ Tierra_del_Fuego.csv

**<h1>1. Carga de datos y armado del dataset</h1>**

In [441]:
# Carga de archivos de datos de chubut
df_chubut = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Chubut.csv')
df_chubut["provincia"]="CHUBUT"
df_chubut.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
9,29901714,Ninfas,entire_home,"4,96 (75)",Puerto Madryn,-42.75380,-65.03627,"['3 camas', '1 dormitorio', '1 baño', '2 camas...",2023-07-02,2023-07-07,NaN,$87 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
103,7461449,Hidden House B&B Tiburon,private_room,"4,76 (85)",Puerto Pirámides,-42.57059,-64.27610,"['3 camas', '1 dormitorio', '1 baño privado', ...",2023-07-01,2023-07-06,$38 USD,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
91,53102190,Moderno monoambiente a 100mts de la playa,entire_home,"4,72 (25)",Rada Tilly,-45.94051,-67.55959,"['1 cama', '1 dormitorio', '1 baño', '1 cama k...",2023-07-01,2023-07-06,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
251,11420435,Cabaña Rural a 25 minutos de Trevelin,entire_home,"5,0 (23)",Trevelin,-43.18388,-71.39454,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-08-16,2023-08-23,$36 USD,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
135,37717690,Casa Siempre Verde,entire_home,"4,67 (6)",Puerto Piramides,-42.56863,-64.27296,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-10,2023-07-15,NaN,$140 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.77073,-65.02947,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
45,29281717,Un lugar de Descanso en Esquel,entire_home,"4,63 (128)",Esquel,-42.92146,-71.32546,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-08,2023-07-13,NaN,$32 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
65,19876278,Departamento con vista al mar en PLENO centro,entire_home,"5,0 (14)",Puerto Madryn,-42.76935,-65.03288,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-11,2023-07-16,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
236,602279361080194562,Cañadon bonito - Cabaña Cipres,entire_home,NaN,Cushamen,-42.11151,-71.44181,"['No hay evaluaciones', '7 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$93 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
87,903126420705158954,Downtown and Beach with seaview,entire_home,Nuevo,Puerto Madryn,-42.76713,-65.03273,"['2 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$34 USD,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [442]:
# Carga de archivos de datos de Neuquen
df_neuquen = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Neuquen.csv')

df_neuquen["provincia"] = "NEUQUEN"

df_neuquen.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
230,40378869,Departamento Céntrico y amplio en Neuquen Capital,entire_home,NaN,Neuquén,-38.957740,-68.061440,"['5 camas', '2 dormitorios', '1 baño', '2 cama...",2023-09-01,2023-09-06,NaN,$28 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
11,831218738602950393,Mono Ambiente céntrico nuevo,entire_home,"4,77 (13)",Neuquén,-38.950840,-68.070320,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$35 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
235,705232918607607106,Hermoso departamento 4 personas,entire_home,NaN,Neuquén,-38.942390,-68.054840,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-04,2023-07-09,NaN,$52 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
16,14788688,Las Violetas 1: alojamiento con vistas,entire_home,"4,76 (26)",Neuquén,-38.943640,-68.051400,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$41 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
15,49819914,Las Violetas 2 - Casa con vistas,entire_home,"4,85 (26)",Neuquén,-38.943110,-68.052640,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-02,2023-07-08,NaN,$45 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
79,897719417897352568,¡Alq. a pasos de Fundación Medica en Cipolletti!,entire_home,Nuevo,Cipolletti,-38.939910,-68.001900,"['1 cama', '1 dormitorio', '1 baño', 'Wifi']",2023-07-01,2023-07-06,NaN,$32 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
232,33031097,Monoambiente 40 mts2 moderno y luminoso,entire_home,"4,74 (60)",Neuquen,-38.954570,-68.077310,"['1 cama', '1 dormitorio', '1 ,5 baños', '1 ca...",2023-12-01,2023-12-06,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
151,824484820620362452,Excelente Departamento Torre de la Costa,entire_home,NaN,Neuquén,-38.958647,-68.023173,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-01,2023-07-06,NaN,$99 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
249,53105603,¡CERCA DE LAS COMPRAS DE HILTON ALTO COMAHUE! ...,entire_home,NaN,Neuquén,-38.942140,-68.064670,"['No hay evaluaciones', '4 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
185,770811620727154311,Casa para 4 personas zona centro,entire_home,NaN,Neuquén,-38.960219,-68.056778,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-02,2023-07-09,NaN,$57 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [443]:
# Carga de archivos de datos de Rio Negro
df_rio_negro = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Rio_Negro.csv')

df_rio_negro["provincia"] = "RIO NEGRO"

df_rio_negro.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
251,4469246,Habitación en Casita de madera y adobe. c/desa...,private_room,"4,68 (114)",San Carlos de Bariloche,-41.073160,-71.488690,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-09,2023-07-14,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
144,609427764930669698,Hermoso monoambiente céntrico,entire_home,"4,97 (32)",San Carlos de Bariloche,-41.138149,-71.305573,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",2023-09-25,2023-09-30,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
161,30204479,La casa de Ni,entire_home,"4,95 (97)",Bariloche,-41.152805,-71.415619,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,$99 USD,$117 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
184,755921214950226681,Casita vegetariana en la naturaleza,entire_home,"4,9 (31)",San Carlos de Bariloche,-41.047005,-71.515422,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,$25 USD,$29 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
46,52169627,Casa frente al lago,entire_home,"4,86 (29)",San Carlos de Bariloche,-41.166040,-71.414610,"['5 camas', '3 dormitorios', '2 baños', 'Estac...",2023-07-01,2023-07-06,NaN,$165 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
25,880037609372703201,Península,entire_home,Nuevo,Bariloche,-41.067385,-71.450768,"['1 cama', '1 dormitorio', '1 baño', 'Wifi']",2023-07-30,2023-08-04,NaN,$180 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
129,16563321,Acogedora cabaña en el lago,entire_home,"4,9 (134)","San Carlos de Bariloche, Río Negro, AR",-41.070180,-71.490940,"['1 cama', '1 dormitorio', '1 cama doble', 'Es...",2023-07-15,2023-07-20,NaN,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
97,48039403,Apartment 2 Pax un Condo between hill and lake,entire_home,"4,88 (89)",San Carlos de Bariloche,-41.124220,-71.366800,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-27,2023-08-01,NaN,$110 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
239,755767521027906453,Casas Calafate VLA,entire_home,"4,9 (10)",Villa La Angostura,-40.767354,-71.644220,"['4 camas', '2 dormitorios', '1 baño', 'Estaci...",2023-07-01,2023-07-06,NaN,$89 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
98,30622116,"Cabaña en el bosque, cerca del centro de Ski",entire_home,"4,84 (69)",San Carlos de Bariloche,-41.149770,-71.407050,"['1 baño', '1 cama queen', 'Estacionamiento gr...",2023-07-07,2023-07-12,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [444]:
# Carga de archivos de datos de Tierra delFuego
df_tierra_del_fuego = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Tierra_del_Fuego.csv')

df_tierra_del_fuego["provincia"] = "TIERRA DEL FUEGO"

df_tierra_del_fuego.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
223,823033004317964906,Departamento Duplex,entire_home,NaN,Ushuaia,-54.806000,-68.32502,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$130 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
31,790475418411671055,Cabañas del Fin del Mundo,private_room,"5,0 (4)",Tolhuin,-54.511280,-67.19323,"['2 camas', '1 dormitorio', '1 baño compartido...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
218,3395040,"Amazing view, 4min away intown.",entire_home,Nuevo,Ushuaia,-54.812930,-68.32646,"['4 camas', '2 dormitorios', '1 baño', 'Wifi',...",2023-07-01,2023-07-06,NaN,$56 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
49,802991806559654665,Vive la magia del Bosque Patagónico,entire_home,"5,0 (14)",Ushuaia,-54.771590,-68.27188,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-02,2023-07-09,NaN,$90 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
34,30937215,Departamento de un dormitorio con sala de estar.,entire_home,"4,81 (47)",Río Grande,-53.770480,-67.71760,"['3 camas', '1 dormitorio', '1 baño', '3 camas...",2023-07-01,2023-07-06,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
63,810966524601056786,Apartamento y spa Ushuaia,entire_home,"4,93 (15)",Ushuaia,-54.793760,-68.23410,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-03,2023-08-09,$59 USD,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
231,762278401905316378,SuperB Apart: gran vista de Beagle,entire_home,"4,78 (18)",Ushuaia,-54.808810,-68.32062,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-29,2023-08-03,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
70,710167445018555343,Apartamento Amundsen 6J - Estudio - Ushuaia,entire_home,"4,87 (23)",Ushuaia,-54.809940,-68.32056,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
156,23973958,"La mejor ubicación en el centro de Ushuaia, pi...",entire_home,"5,0 (149)",Ushuaia,-54.804787,-68.30304,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-09-15,2023-09-20,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
152,730606158433979341,Terra San Martín,entire_home,"4,97 (30)",Ushuaia,-54.809440,-68.31436,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-08-01,2023-08-07,NaN,$81 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [445]:
# Verificacion de columnas para concatenar
(df_chubut.columns == df_neuquen.columns) & (df_rio_negro.columns == df_tierra_del_fuego.columns) & (df_chubut.columns == df_tierra_del_fuego.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [446]:
# Concatenacion de los dataframes separados por provincias en un unico archivo
df_alquileres = pd.concat([df_chubut, df_neuquen, df_rio_negro, df_tierra_del_fuego])
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
132,41321385,6 - Pura tranquilidad en el corazón de Neuquén,private_room,"4,5 (4)",Neuquén,-38.953410,-68.106270,"['2 baños compartidos', '1 cama doble', 'Estac...",2023-07-01,2023-07-06,NaN,$40 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
135,891565292344373926,Departamento Alto Neuquén,entire_home,Nuevo,MDJ,-38.942705,-68.055434,"['2 camas', '2 dormitorios', '1 baño', 'Estaci...",2023-07-02,2023-07-08,$40 USD,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
268,39950619,"Mi casa, tú casa...",entire_home,"5,0 (18)",Esquel,-42.917590,-71.329970,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
263,31021085,Un lugar en contacto con la naturaleza,entire_home,"4,5 (10)",Lago Puelo,-42.080110,-71.610830,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
136,831164056448451302,Departamento con balcón,entire_home,NaN,Neuquén,-38.925274,-68.052651,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-17,2023-07-22,NaN,$37 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
126,24481424,Patagonia Vantage Point,entire_home,"4,96 (191)",San Carlos de Bariloche,-41.133630,-71.318730,"['1 cama', '1 dormitorio', '1 ,5 baños', '1 ca...",2023-09-30,2023-10-05,NaN,$79 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
247,889382807675939102,Acogedor departamento de 2 amb,entire_home,Nuevo,San Carlos de Bariloche,-41.122310,-71.369530,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-07-05,2023-07-10,NaN,$62 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
123,805618749351197751,"Cabaña El Remanso, Bahía Manzano (AND12)",entire_home,"5,0 (6)",Puerto Manzano,-40.806570,-71.594620,"['5 camas', '2 dormitorios', '2 baños', '1 cam...",2023-09-02,2023-09-07,NaN,$90 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
210,630333448514430277,Hermoso bed and breakfast frente al Atlántico,private_room,NaN,Puerto Madryn,-42.748570,-65.038810,"['No hay evaluaciones', '6 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
194,30380680,Casa de Cecilia,private_room,"4,89 (70)",Esquel,-42.914520,-71.316790,"['2 camas', '1 dormitorio', '2 baños compartid...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [447]:
# Reseteo del index, permitiendo duplicados porque aun no sabemos si son unicos los valores
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= True)
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= False)
df_alquileres.reset_index(drop=True,inplace=True)

*Verificacion de filas de los dataframes*

In [448]:
# Verificacion de la cantidad de filas de cada dataframe
print("La cantidad de filas del dataframe de Chubut es:"+str(df_chubut.shape[0]))
print("La cantidad de filas del dataframe de Neuquen es:"+str(df_neuquen.shape[0]))
print("La cantidad de filas del dataframe de Rio Negro es:"+str(df_rio_negro.shape[0]))
print("La cantidad de filas del dataframe de Tierra del fuego es:" + str(df_tierra_del_fuego.shape[0]))
print("La cantidad de filas del dataframe unificado de alquileres es:" + str(df_alquileres.shape[0]))


La cantidad de filas del dataframe de Chubut es:270
La cantidad de filas del dataframe de Neuquen es:270
La cantidad de filas del dataframe de Rio Negro es:270
La cantidad de filas del dataframe de Tierra del fuego es:270
La cantidad de filas del dataframe unificado de alquileres es:1080


*Verificacion de las columnas de los dataframes*

In [449]:
# Verificacion de la cantidad de columnas de cada dataframe
print("La cantidad de columnas del dataframe de Chubut es:" +
      str(df_chubut.shape[1]))
print("La cantidad de columnas del dataframe de Neuquen es:" +
      str(df_neuquen.shape[1]))
print("La cantidad de columnas del dataframe de Rio Negro es:" +
      str(df_rio_negro.shape[1]))
print("La cantidad de columnas del dataframe de Tierra del fuego es:" +
      str(df_tierra_del_fuego.shape[1]))
print("La cantidad de columnas del dataframe unificado de alquileres es:" +
      str(df_alquileres.shape[1]))

La cantidad de columnas del dataframe de Chubut es:18
La cantidad de columnas del dataframe de Neuquen es:18
La cantidad de columnas del dataframe de Rio Negro es:18
La cantidad de columnas del dataframe de Tierra del fuego es:18
La cantidad de columnas del dataframe unificado de alquileres es:18


*Verificacion general e inicial del dataframe unificado*

In [450]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1080 non-null   int64  
 1   name              1080 non-null   object 
 2   category          1080 non-null   object 
 3   rating            886 non-null    object 
 4   city              1076 non-null   object 
 5   latitude          1080 non-null   float64
 6   longitude         1080 non-null   float64
 7   characteristics   1080 non-null   object 
 8   check_in          1080 non-null   object 
 9   check_out         1080 non-null   object 
 10  price_discounted  201 non-null    object 
 11  price_original    1080 non-null   object 
 12  price_qualifier   1080 non-null   object 
 13  cant_resultados   1080 non-null   int64  
 14  fecha_scrapping   1080 non-null   object 
 15  fecha_cambio_bna  1080 non-null   object 
 16  cambio_bna        1080 non-null   float64


In [451]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.describe()

,id_alojamiento,latitude,longitude,cant_resultados,cambio_bna
count,1.080000e+03,1080.000000,1080.000000,1080.000000,1080.000000
mean,3.685341e+17,-44.426201,-68.723924,824.500000,239.919355
std,3.903051e+17,6.164758,2.159715,304.115744,0.000000
min,4.468310e+05,-54.844914,-71.688000,298.000000,239.919355
25%,3.529220e+07,-47.854503,-71.310280,824.500000,239.919355
50%,5.408344e+07,-42.003780,-68.305610,1000.000000,239.919355
75%,7.872495e+17,-39.609811,-68.049328,1000.000000,239.919355
max,9.198428e+17,-38.874370,-62.920960,1000.000000,239.919355


In [452]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.duplicated(keep="first").sum()

51

In [453]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.isna().sum()

id_alojamiento        0
name                  0
category              0
rating              194
city                  4
latitude              0
longitude             0
characteristics       0
check_in              0
check_out             0
price_discounted    879
price_original        0
price_qualifier       0
cant_resultados       0
fecha_scrapping       0
fecha_cambio_bna      0
cambio_bna            0
provincia             0
dtype: int64

*Conclusiones iniciales*
<br>
+ El dataframe unificado, coincide en cantidad de filas y columnas correspondientes a los dataframes individuales
+ Debemos de corregir algunos tipos de datos de columnas
+ Tenemos algunas columnas para reescalar
+ Tenemos filas duplicadas para revisar
+ Tenemos valores nulos para revisar, en principio para las columnas rating y price_discounted

**<h1>2. Análisis de duplicados</h1>**

*Analisis y tratamiento de duplicados global*

In [454]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres.duplicated(keep="first").sum()

51

In [455]:
#  Listo los valores duplicados
df_alquileres[df_alquileres.duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
54,40755147,La Amarilla Casa de Huéspedes (con cocina) Rut...,entire_home,"4,93 (27)",Epuyén,-42.227509,-71.362289,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
72,586216477715336949,Encantador dpto. con vista al mar en Puerto Ma...,entire_home,"5,0 (11)",Puerto Madryn,-42.771633,-65.029068,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-07-06,2023-07-11,NaN,$131 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
90,37836757,Rincón del Sol,entire_home,"4,4 (5)",Puerto Madryn,-42.751650,-65.053520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
113,736253141994173324,Refugio Ciprés con tina de agua caliente,entire_home,"5,0 (4)",Trevelin,-42.968260,-71.601320,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
127,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
128,644043911246503372,"Los Molles, casa familiar en entorno natural.",entire_home,NaN,Puerto Madryn,-42.735607,-65.100117,"['3 camas', '2 dormitorios', '1 baño']",2023-07-01,2023-07-07,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
129,883174783781736766,Espacio 2 - Rada Tilly,entire_home,Nuevo,Rada Tilly,-45.925810,-67.575650,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
138,53298317,Mar de Olivillos Planta alta,entire_home,"5,0 (12)",Puerto Piramides,-42.569200,-64.273610,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
162,31081731,"Domos Puelo, para dos, con jacuzzi",entire_home,"4,98 (40)",Lago Puelo,-42.041570,-71.592370,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-08,2023-07-15,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
182,50704005,Hostel patagónico con vista 360 de la Cordillera,shared_room,"4,86 (7)",Trevelin,-43.071450,-71.454920,"['20 camas', '4 dormitorios', '3,5 baños compa...",2023-07-01,2023-07-06,NaN,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [456]:
# Elimino los duplicados, que coinciden en toda la fila
df_alquileres.drop_duplicates(keep="first", inplace=True)
df_alquileres.shape

(1029, 18)

*Analisis y tratamiento de duplicados por columna*

+ Columna 1: id_alojamiento

In [457]:
# Busco si hay duplicados
df_alquileres['id_alojamiento'].duplicated(keep='first').sum()

1

In [458]:
# Identifico filas con esos valores duplicados
df_alquileres[df_alquileres['id_alojamiento'].duplicated(keep='first')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [459]:
# Muestro las filas que corresponden a los valores duplicados, todas
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


-> Analisis de los valores de las filas duplicadas.

In [460]:
#Verificamos los valores 
df_alquileres.loc[647]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [461]:
# Verificamos los valores
df_alquileres.loc[648]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [462]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[647]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi']"

In [463]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[648]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi', 'Calificado con 5 de 5 en limpieza']"

*ACCION:*
<BR>
*Vemos que seria el mismo registro con excepcion de la serie characteristics, donde tiene agregada comentario de calificacion*
<br>
*Como consideramos que cada valor de id_alojamiento, deberia ser unico, eliminamos el que consideramos repetido y menos se adecua a los valores esperados por serie*

In [464]:
# Descartamos el registro que no tiene agregado el comentario de la calificacion
df_alquileres.drop([648], inplace=True)
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


+ Columna 2: name

In [465]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['name'].duplicated(keep="first").sum()

1

In [466]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres[df_alquileres['name'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.1333,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [467]:
df_alquileres[df_alquileres['name'].str.contains('Vista al Lago en el Bosque - Hab Compartida')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
712,797077259992479162,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13455,-71.36807,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-11,2023-07-18,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13330,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


*ACCION:*
<br>
*En este caso vemos que los nombre coinciden pero, pueden corresponder a diferentes lugares, de hecho, estan sobre diferentes calles, segun sus coordenadas.*
<br>
*Por tal motivo los dejamos a ambos registros*

+ Columna 3: category

In [468]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['category'].duplicated(keep="first").sum()

1025

In [469]:
# Listo los duplicados segun category
df_alquileres[df_alquileres['category'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
224,903553078458767448,Alojamiento Sabio,entire_home,Nuevo,Lago Puelo,-42.066230,-71.599730,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$43 USD,$53 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
713,52993593,Habitación privada en San Martín de Los Andes ...,private_room,"4,96 (27)",San Martin de los Andes,-40.133560,-71.297480,"['1 cama', '1 dormitorio', '2 baños compartido...",2023-07-01,2023-07-06,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
489,49179716,Moderno departamento para 4 personas,entire_home,"4,67 (3)",Cipolletti,-38.948870,-67.983440,"['4 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-17,2023-08-22,NaN,$47 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
691,554319517238694179,Cabaña Raulí para 8 con jacuzzi,entire_home,"4,94 (17)",San Carlos de Bariloche,-41.145020,-71.412740,"['7 camas', '4 dormitorios', '1 ,5 baños', '2 ...",2023-09-05,2023-09-11,$157 USD,$185 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
357,913610159893611362,Apart Rivadavia 340,entire_home,Nuevo,Neuquén,-38.953180,-68.054620,"['2 camas', 'Monoambiente', '1 baño', 'Wifi']",2023-07-02,2023-07-07,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
740,6993416,Cabaña Lago Bahía Protegida Costa Lago,entire_home,"4,92 (100)",San Carlos de Bariloche,-41.098340,-71.446220,"['6 camas', '2 dormitorios', '2 baños', 'Estac...",2023-08-12,2023-08-17,NaN,$300 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
262,903635543144861390,Departamento a 1 cuadra del mar,entire_home,Nuevo,Puerto Madryn,-42.756760,-65.037320,"['3 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$42 USD,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
56,845848067167341979,Almacén de Amor,private_room,"4,5 (4)",Esquel,-42.919560,-71.331530,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
803,22985648,Casa de montaña c/ vista al lago - Radales,entire_home,"4,95 (60)",San Carlos de Bariloche,-41.075790,-71.464440,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-08-20,2023-08-25,NaN,$109 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
244,808583268258987047,Alojamiento granjero en chacra Prado Verde,private_room,NaN,Gaiman,-43.315467,-65.522409,"['5 camas', '3 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$20 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que los valores de categoria se pueden repetir para los diferentes alquileres*
<br>
*por lo cual, no realizamos ninguna accion en base a esta columna*

+ Columna 4: rating

In [470]:
# Verifico los duplicado
df_alquileres['rating'].duplicated(keep="first").sum()

564

In [471]:
# Listamos algunas filas, repetidas por rating
df_alquileres[df_alquileres['rating'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
262,903635543144861390,Departamento a 1 cuadra del mar,entire_home,Nuevo,Puerto Madryn,-42.756760,-65.037320,"['3 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$42 USD,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.803600,-68.321320,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
328,699352390023248021,Duplex,entire_home,Nuevo,Neuquén,-38.944580,-68.074130,"['2 camas', '2 dormitorios', '2 baños', 'Estac...",2023-07-01,2023-07-06,$36 USD,$45 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
231,638343989984922164,Luminoso depto a una cuadra del mar Puerto Madryn,entire_home,"4,93 (14)",Puerto Madryn,-42.756570,-65.037170,"['3 camas', '1 dormitorio', '1 baño', '2 camas...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
197,831753472693247654,"Mamma 2, Esquel.",entire_home,"5,0 (8)",Esquel,-42.908410,-71.298020,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
365,906751625852578545,Departamento reciclado,entire_home,Nuevo,Neuquén,-38.949245,-68.033369,"['2 camas', '1 dormitorio', '1 baño', 'Wifi']",2023-07-08,2023-07-13,$26 USD,$33 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
383,28084820,Departamento cerca del Río Grande,entire_home,"5,0 (3)",Neuquén,-38.975090,-68.056720,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$51 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
818,889581630642800275,Beagle Suites - Apt del Canal,entire_home,Nuevo,Ushuaia,-54.843030,-68.380370,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-08-02,2023-08-07,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
145,41171673,Las Condalias - Pto Madryn. Luminoso y tranquilo.,entire_home,"4,79 (14)",Biedma,-42.740400,-65.102580,"['8 camas', '3 dormitorios', '2 baños', '2 cam...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
894,831395831267634973,Dpto Senderos de Andorra,entire_home,NaN,Ushuaia,-54.773356,-68.273620,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-21,2023-07-28,NaN,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [472]:
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

*ACCION:*
<br>
*Entendemos que los valores de rating se pueden repetir para los diferentes alquileres*
<br>
*Por lo cual, dejamos los registros retpetidos segun esta columna*

+ Columna 5: city

In [473]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['city'].duplicated(keep="first").sum()

947

In [474]:
# Listamos repetidos, por city
df_alquileres[df_alquileres['city'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,NaN,Puerto Madryn,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
2,800749301724167838,Departamento Bella Vista,entire_home,"5,0 (6)",Puerto Madryn,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,"4,91 (70)",Puerto Madryn,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
4,916738743930751566,Front Beach,entire_home,Nuevo,Puerto Madryn,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
5,27877802,Luxury top 11th floor with the best 360 views,entire_home,"4,96 (95)",Puerto Madryn,-42.768330,-65.032100,"['4 camas', '2 dormitorios', '2,5 baños', '1 c...",2023-07-06,2023-07-11,NaN,$122 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,771725397497773450,Casa Fueguina en Centro,entire_home,"4,53 (17)",Ushuaia,-54.803310,-68.302580,"['3 camas', '2 dormitorios', '1 baño', 'Wifi',...",2023-08-14,2023-08-19,$60 USD,$76 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1076,26215568,Hospedaje UshuaiAndo,entire_home,"4,82 (50)",Ushuaia,-54.800790,-68.304230,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-23,2023-07-28,$45 USD,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1078,51921207,KREN I,entire_home,"4,85 (26)",Ushuaia,-54.806170,-68.317800,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-10,2023-07-15,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna city, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 6: latitude

In [475]:
# Verificamos los duplicados
df_alquileres['latitude'].duplicated(keep="first").sum()

36

In [476]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['latitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.80543,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
443,53105728,LOFT RESIDENCIAL ALTO COMAHUE DE COMPRAS!!,entire_home,NaN,Neuquén,-38.94145,-68.063080,"['No hay evaluaciones', '2 camas', 'Monoambien...",2023-07-01,2023-07-06,NaN,$20 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.75560,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
536,39606454,AMUEBLADOS COMAHUE - Temp. Growing - 3pax,entire_home,NaN,Neuquén,-38.94389,-68.054130,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$70 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
791,4469246,Habitación en Casita de madera y adobe. c/desa...,private_room,"4,68 (114)",San Carlos de Bariloche,-41.07316,-71.488690,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-09,2023-07-14,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
364,810083074082367782,Departamento luminoso centrico,entire_home,NaN,Neuquén,-38.94956,-68.054970,"['No hay evaluaciones', '1 cama', 'Monoambient...",2023-10-02,2023-10-07,NaN,$22 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
296,661388148387143050,Apart céntrico 2 dorm c/ balcon cochera opci...,entire_home,"5,0 (6)",Neuquén,-38.95141,-68.061740,"['3 camas', '2 dormitorios', '1 ,5 baños', 'Wi...",2023-07-01,2023-07-08,NaN,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
318,34539785,Dpto B° Lejos de Buenos Aires.,entire_home,"4,91 (69)",Neuquén,-38.95141,-68.114000,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$47 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
717,49546396,Departamento para 2. Excelente ubicación.,entire_home,"4,89 (53)",San Carlos de Bariloche,-41.13455,-71.299200,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",2023-07-07,2023-07-13,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1024,906355891628807276,Grand House,entire_home,Nuevo,Ushuaia,-54.80678,-68.315870,"['5 camas', '2 dormitorios', '2 baños', 'Wifi'...",2023-07-03,2023-07-09,$140 USD,$175 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [477]:
# Vemos un ejemplo
df_alquileres[df_alquileres['latitude'] == -54.803600]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
934,31825003,Departamento con inmejorable ubicación en Ushuaia,entire_home,"4,9 (108)",Ushuaia,-54.8036,-68.30374,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.8036,-68.32132,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna latitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 7: longitude

In [478]:
# Verificamos los duplicados
df_alquileres['longitude'].duplicated(keep="first").sum()

20

In [479]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['longitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1044,51921353,KREN II,entire_home,"4,77 (30)",Ushuaia,-54.807290,-68.316100,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
243,909336244153347770,Bella Vista 2,entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['4 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$52 USD,$65 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1050,828090107927300389,GRUshuaia,entire_home,NaN,Ushuaia,-54.815088,-68.329166,"['No hay evaluaciones', '1 cama', 'Monoambient...",2023-09-13,2023-09-18,$51 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.755600,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
15,767503846951993933,Brisa de Mar - Puerto Madryn,entire_home,"4,95 (22)",Puerto Madryn,-42.771634,-65.029071,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-08-07,2023-08-12,$34 USD,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
534,41228116,Mono tipo LOFT ALTO COMAHUE SHOPPING!!,entire_home,NaN,Neuquén,-38.943410,-68.064100,"['No hay evaluaciones', '2 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$30 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
531,833878451226342434,El lugar de Ana - Am. Comahue,entire_home,NaN,Neuquén,-38.953065,-68.065207,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-02,2023-07-08,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
905,579038522149900540,Departamento con hermosa vista a la Bahía,entire_home,"4,81 (21)",Ushuaia,-54.807163,-68.318520,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-03,2023-07-08,NaN,$77 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
86,51311028,"Amplio Monoambiente con estacionamiento,cerrado.",entire_home,"4,55 (33)",Esquel,-42.915770,-71.307150,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$20 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
527,32223913,Mono-Leloir AMUEBLADOS COMAHUE,entire_home,NaN,Neuquén,-38.941620,-68.056090,"['2 camas', 'Monoambiente', '0 baños', '1 sofá...",2023-07-01,2023-07-06,NaN,$53 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [480]:
# Vemos un ejemplo
df_alquileres[df_alquileres['longitude'] == -65.029470]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
70,20364091,Departamento Solmar,entire_home,"4,82 (137)",Puerto Madryn,-42.76963,-65.02947,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-15,2023-10-20,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.77073,-65.02947,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que la columna longitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columan 8: characteristics

In [481]:
# Verificamos los duplicados
df_alquileres['characteristics'].duplicated(keep="first").sum()

243

In [482]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['characteristics'].duplicated(
    keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
523,46399823,"Apartamento a un kilómetro del centro, Cipolle...",entire_home,"5,0 (4)",Cipolletti,-38.937860,-67.979230,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$69 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
560,876704382378753640,Encantadora cabaña rústica con acceso al lago,entire_home,Nuevo,San Carlos de Bariloche,-41.167240,-71.416200,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-01,2023-07-06,NaN,$69 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
946,787731307979306505,Alquilo habitación muy espaciosa de dos camas.,private_room,"4,91 (11)",DWC,-54.819210,-68.336300,"['2 camas', '1 dormitorio', 'Wifi', 'Cocina', ...",2023-07-01,2023-07-06,$20 USD,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
695,27782581,Depto Soñado en San Martín de los Andes (centro),entire_home,"4,96 (108)",San Martin de los Andes,-40.152400,-71.351520,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,$62 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
118,887853891920643116,"Luma, tu lugar feliz en el sur",entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$34 USD,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
661,33512769,Las Liebres Doble - D.T. 48/19,entire_home,"4,92 (90)",San Carlos de Bariloche,-41.127170,-71.352060,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-09-26,2023-10-01,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1041,762278401905316378,SuperB Apart: gran vista de Beagle,entire_home,"4,78 (18)",Ushuaia,-54.808810,-68.320620,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-29,2023-08-03,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
120,30564017,Casita Indigo,entire_home,"4,97 (63)",Lago Puelo,-42.079910,-71.611130,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-11-01,2023-11-06,NaN,$26 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
186,689975266751251330,Hermoso departamento a pocos metros del mar,entire_home,"5,0 (7)",Puerto Madryn,-42.762380,-65.036084,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-07-01,2023-07-06,NaN,$49 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
193,839910728991938719,"""El Rancho"" Sentirte en casa",entire_home,NaN,Lago Puelo,-42.034017,-71.545491,"['3 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que la columna characteristics, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 9: check_in

In [483]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_in'].duplicated(keep="first").sum()

920

*ACCION:*
<br>
*Entendemos que la columna check_in, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 10: check_out

In [484]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_out'].duplicated(keep="first").sum()

916

*ACCION:*
<br>
*Entendemos que la columna check_out, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 11: price_discounted

In [485]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_discounted'].duplicated(keep="first").sum()

938

*ACCION:*
<br>
*Entendemos que la columna price_discounted, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 12: price_original

In [486]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_original'].duplicated(keep="first").sum()

868

*ACCION:*
<br>
*Entendemos que la columna price_original, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 13: price_qualifier

In [487]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_qualifier'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna price_qualifier, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 14: cant_resultados

In [488]:
# Verificamos los duplicados
df_alquileres['cant_resultados'].duplicated(keep="first").sum()

1026

*ACCION:*
<br>
*Entendemos que la columna cant_resultados, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

Columna 15: fecha_scrapping

In [489]:
# Verificamos los duplicados
df_alquileres['fecha_scrapping'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_scrapping, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 16: fecha_cambio_bna

In [490]:
# Verificamos los duplicados
df_alquileres['fecha_cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 17: cambio_bna

In [491]:
# Verificamos los duplicados
df_alquileres['cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 18: provincia

In [492]:
# Verificamos los duplicados
df_alquileres['provincia'].duplicated(keep="first").sum()

1024

*ACCION:*
<br>
*Entendemos que la columna provincia, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

*OBSERVACION GENERAL*
<br>
*En general entendemos que la mayoria de las columnas podrian tener valores repetidos evaluandolas individualmente.*
<br>
*Pero si consideramos que la variable id_alojamiento identifica de manera biunivoca un alojamiento.*
<br>
*Podemos decir que no es lo mismo, un duplicado en la columna id_alojamiento(que no deberia tener valores repetidos), que un duplicado en la columna price_original,*
<br>
*(que podria perfectamente tener valores repetidos en la seria).*


**<h1>3. Análisis de datos faltantes y limpieza</h1>**

+ Mostrar el porcentaje de datos faltantes de cada columna

In [493]:
# Verificacion inicial del dataset
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

*En primera instancia observamos que solo nos quedan dos columnas con datos faltantes: rating y price_discounted*
<br>
*Para rating, tenenmos un 81.90 % de faltantes (842 de 1028).*
<br>
*Para rating, tenenmos un 18.58 % de faltantes (191 de 1028).*


+ ¿Cuáles son las 3 columnas con más datos faltantes?

*En nuestra observacion inicial, en principio tenemos dos columnas con datos faltantes solamente: rating y price_discounted*

**LIMPIEZA DE DE DATOS POR COLUMNAS**

In [494]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

+ Columna 0: id_alojamiento

In [495]:
df_alquileres['id_alojamiento'].nunique()

1028

In [496]:
df_alquileres['id_alojamiento'].isna().sum()

0

+ Columna 1: name

In [497]:
sorted(df_alquileres['name'].unique())

['"Alegria" vistas de ensueño en un lugar para enamorarse',
 '"Bathroom with LakeView Place and Retreat"',
 '"Casita" a estrenar, 21 m2 de luz',
 '"Cerros Nevados" departamento tipo loft.',
 '"Cipresales"     Cabaña  Villa la Angostura  Nqn',
 '"Doña Cota" - Amueblados Comahue',
 '"El Rancho" Sentirte en casa',
 '"La Encantada" en Villa los Coihues',
 '"La Posadita" Casa de Montaña, Miralejos',
 '"La casa de Cielo "',
 '"Los Amancays "\nCabaña de montaña',
 '"Tiny House de Ensueño"',
 '"Ty Bychan" \nLa Casita. Entre raíces galesas.',
 '0 - Pura Tranquilidad en el Corazón de Neuquén',
 '1- Pura Tranquilidad en el Corazón de Neuquén',
 '11-E Espectacular departamento Alto Neuquen',
 '2 - Pura tranquilidad en el corazón de Neuquén',
 '2 Amigos de viaje en un lugar comodo',
 '2 Habitaciones. 4 Personas en Casa con Pileta',
 '2 Personas. Lugar súper cómodo',
 '2 ambientes en el centro',
 '2x1 Central Family',
 '3 dormitorios. cama matrimonial + 3 de una plaza',
 '3- Pura tranquilidad en el 

+ Columna 2: category

In [498]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['category'].unique()

array(['entire_home', 'private_room', 'shared_room'], dtype=object)

In [499]:
# Verificamos que no tienen  na
df_alquileres['category'].isna().sum()

0

In [500]:
# Cambiamos el tipo de la serie
df_alquileres['category'] = df_alquileres['category'].astype('category')

+ Columna 3: rating

In [501]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

In [502]:
# Observamos que existen valores na y uno que seria "Nuevo"
# Reemplazamos el valor Nuevo por na
mask = df_alquileres['rating'].str.lower().str.contains('nuevo', na = False)
df_alquileres.loc[mask, 'rating'] = np.NaN

In [503]:
# Tratamiento de la columna rating
# Separamos los valores correspondientes para limpiarlos
df_rating= df_alquileres['rating'].str.split(pat="[ ]",expand=True).add_prefix('Columna')
# Renombramos las columnas
df_rating.rename(columns={"Columna0": "rating", "Columna1": "votaciones"}, inplace=True)

In [504]:
# Limpiamos los valores para la columna rating
df_rating['rating'] = df_rating['rating'].replace(',','.')

In [505]:
# Reemplazamos la columna por punto para la columna rating y lo cambiamos de tipo
df_rating['rating'] = df_rating['rating'].str.replace(',', '.')
df_rating['rating'] = df_rating['rating'].astype('float64')

In [506]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      726 non-null    float64
 1   votaciones  726 non-null    object 
dtypes: float64(1), object(1)
memory usage: 24.1+ KB


In [507]:
# Limpiamos la columna de votaciones para cambiarle el tipo
df_rating['votaciones'] = df_rating['votaciones'].str.replace('(', '').str.replace(')', '')
df_rating['votaciones'] = np.floor(pd.to_numeric(df_rating['votaciones'], errors='coerce')).astype('Int64')


In [508]:
#  Rellenamos los valores para las columnas rating y votaciones
# Rellenamos con un valor promedio, pero establecemos que no tiene votaciones
media_rating = df_rating['rating'].mean()
media_votaciones = np.floor(df_rating['votaciones'].mean())
df_rating.fillna({'rating':media_rating,'votaciones':media_votaciones},inplace= True)


In [509]:
# Enriquecemos el dataframe de alquileres con las columnas de rating y votaciones
df_alquileres= df_alquileres.merge(df_rating,how='left',left_index=True,right_index=True,suffixes=('', '_y'))
df_alquileres[['rating_y','votaciones']].head()


,rating_y,votaciones
0,4.846171,44
1,4.846171,44
2,5.000000,6
3,4.910000,70
4,4.846171,44


In [510]:
# Actualizamos las columnas del dataframe rating y votaciones
df_alquileres['rating']= df_alquileres['rating_y']
df_alquileres = df_alquileres.drop('rating_y',axis=1)
df_alquileres.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   object  
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 non-nu

+ Agregamos la columna: best_reviews

In [511]:
df_alquileres['best_reviews'] = np.where(df_alquileres['rating']==5.0,True,False)
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
643,33972220,PUERTO LILIPUT -Exclusive Lodge-Cabaña con costa,entire_home,4.980000,San Carlos de Bariloche,-41.162900,-71.415280,"['6 camas', '3 dormitorios', '2 baños', '1 cam...",2023-09-07,2023-09-12,NaN,$294 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO,44,False
490,47372143,Dúplex amoblados de dos dormitorios,entire_home,4.846171,Confluencia,-38.954000,-68.203960,"['4 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$100 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN,44,False
116,23014614,Apart Shape,entire_home,4.380000,Comodoro Rivadavia,-45.794890,-67.486000,"['2 camas', '1 dormitorio', '1 baño', '2 camas...",2023-07-01,2023-07-06,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,13,False
256,53308828,Lo de Gustavo Villa Ayelen,entire_home,4.846171,Esquel,-42.918510,-71.355190,"['5 camas', '2 dormitorios', '2,5 baños']",2023-07-01,2023-07-06,NaN,$115 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
88,884041797814332468,hermosa casa,entire_home,4.846171,Esquel,-42.920980,-71.331340,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-23,2023-07-28,NaN,$60 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
204,656442,Hidden House B&B - Mojarrita,private_room,4.690000,Biedma Department,-42.571500,-64.275750,"['1 cama', '1 dormitorio', '1 baño privado', '...",2023-07-01,2023-07-06,$38 USD,$78 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,101,False
573,48316755,Beautiful studio with great views,entire_home,5.000000,San Carlos de Bariloche,-41.122440,-71.389390,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-18,2023-07-23,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO,55,True
124,644043911246503372,"Los Molles, casa familiar en entorno natural.",entire_home,4.846171,Puerto Madryn,-42.735607,-65.100117,"['3 camas', '2 dormitorios', '1 baño']",2023-07-01,2023-07-07,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
293,742686726143047393,Studio cerca del centro Neuquen,entire_home,4.750000,Neuquén,-38.943883,-68.046642,"['2 camas', 'Monoambiente', '1 baño', 'Estacio...",2023-07-01,2023-07-06,NaN,$24 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN,4,False
637,48039403,Apartment 2 Pax un Condo between hill and lake,entire_home,4.880000,San Carlos de Bariloche,-41.124220,-71.366800,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-27,2023-08-01,NaN,$110 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO,89,False


In [512]:
# Porcentaje de moejores reviews
mejores = df_alquileres[df_alquileres["best_reviews"]]['best_reviews'].count()
no_mejores = df_alquileres[~df_alquileres["best_reviews"]]['best_reviews'].count()
porc = mejores /(mejores+no_mejores)*100
print(f"La cantidad de mejores reseñas es de :{mejores} y corresponde a un {porc} %.")

La cantidad de mejores reseñas es de :161 y corresponde a un 15.66147859922179 %.


+ Columna 4: city

In [513]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['city'].unique()

array(['Puerto Madryn', 'Comodoro Rivadavia', 'Trelew', 'Esquel',
       'Puerto Piramides', 'Puerto Pirámides', 'Trevelin', 'Cholila',
       'Epuyén', 'Playa Unión', 'Lago Puelo', 'Playa Union', 'Rada Tilly',
       'Rawson', 'Biedma', 'Esquel,', 'Gaiman', 'El Hoyo', 'Cushamen',
       'Playa Magagna', 'Camarones', 'Biedma Department',
       'Gobernador Costa', 'Corcovado, Chubut', 'Neuquén', 'Neuquen',
       'BVH', 'Cipolletti', 'MDJ', 'Plottier', 'Confluencia',
       'Centenario', nan, 'Q8302ATE', 'JKB', 'Las Perlas', 'KCN',
       'Q8300MDJ', 'Confluencia Department', 'San Martín de Los Andes',
       'BGJ', 'Villa Traful', 'Villa La Angostura',
       'San Carlos de Bariloche', 'Bariloche', 'San Martin de los Andes',
       'Dina Huapi', 'Lácar', 'San Martín de los Andes', 'La Villa',
       'San Carlos de Bariloche-  Península de San Pedro',
       'Villa la Angostura ', 'Los Lagos', 'Puerto Manzano', 'Bariloche ',
       'San Carlos de Bariloche, Río Negro, AR', 'Viedma',
  

In [514]:
# Como vemos que varias ciudades son la misma pero con alguna diferencia tipografica, tratamos de normalizar los valores
df_alquileres['city'] = df_alquileres['city'].str.strip().str.upper()
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
0,855050336912355203,North Beach,entire_home,4.846171,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,4.846171,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
2,800749301724167838,Departamento Bella Vista,entire_home,5.000000,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6,True
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.910000,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70,False
4,916738743930751566,Front Beach,entire_home,4.846171,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False


In [515]:
# Quitamos los tildes
df_alquileres['city'] = df_alquileres['city'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews
0,855050336912355203,North Beach,entire_home,4.846171,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,4.846171,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False
2,800749301724167838,Departamento Bella Vista,entire_home,5.000000,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6,True
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.910000,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70,False
4,916738743930751566,Front Beach,entire_home,4.846171,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False


In [516]:
# Recopilamos los datos de ciudades que tienen lugeres turisicos, segun la provincia, para tratar de agruparlas
aux_ciudades = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Provincias_Ciudades.csv',sep= ';',encoding='latin1')
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.upper().str.strip()
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.normalize(
    'NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
aux_ciudades.head()

,ciudad,provincia
0,ALTO RIO SENGUER,CHUBUT
1,BAHIA BUSTAMANTE,CHUBUT
2,CAMARONES,CHUBUT
3,CHOLILA,CHUBUT
4,COMODORO RIVADAVIA,CHUBUT


In [517]:
# Creamos una columna city2, que va a a guardar los valores resultantes para city
df_alquileres['city2'] = df_alquileres['city']


# Parametros para los cuales buscar la mejor aproximacion por fuzzy
provincias = ['TIERRA DEL FUEGO', 'NEUQUEN', 'RIO NEGRO', 'CHUBUT']
# Valres de match que aceptamos
aprobaciones = [80, 80, 80, 80]

# rr=0;
# iter=0
# Iteramos por indice, para provincias y aprobaciones
for indice in range((len(provincias))):
    # Iteramos  por cada ciudad segun la provincia
    for _, row in df_alquileres.loc[df_alquileres['provincia']==provincias[indice]].iterrows():
        ciudad_probar = str(row['city'])
        #aloj=-1
        #iter+=1
        max_coincidencia=0
        encontro = False
        # Recorremos la lista de ciudades de la provincia recabados para verificar
        for valor_correcto in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:            
            # Obtenemos el ratio de coincidencia
            coincidencia = fuzz.ratio(ciudad_probar, valor_correcto)
            #print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto} de {provincias[indice]}. Resultdo {coincidencia}")
            # Si el puntaje de la coincidencia es mayor que la vigente
            if coincidencia > max_coincidencia:
                # Guardar el valor correcto
                correccion = valor_correcto
                # Reemplazar el puntaje
                max_coincidencia = coincidencia                
                if (coincidencia > max_coincidencia) & (max_coincidencia >= aprobaciones[indice]):
                    # Guardar el valor correcto
                    correccion = valor_correcto
                    # Reemplazar el puntaje
                    max_coincidencia = coincidencia
                    # Reemplazo por el mejor valor aproximado, en la columna auxiliar
                    df_alquileres['city2']=correccion
                    # Aviso que encontre algo
                    encontro = True
        # Si no encontre una coincidencia suficiente, cambio de macheo
        if( not encontro):
            # Recorremos la lista de departamentos
            max_coincidencia_parcial = 0            
            for valor_correcto_parcial in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:
                # Obtenemos el ratio de coincidencia
                coincidencia_parcial = fuzz.partial_ratio(ciudad_probar, valor_correcto_parcial)
                # print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto_parcial} de {provincias[indice]}. Resultdo {coincidencia_parcial}")
                # Si el puntaje de la coincidencia es mayor que la vigente
                if coincidencia_parcial > max_coincidencia_parcial:
                    # Guardar el valor correcto
                    correccion_parcial = valor_correcto_parcial
                    # Reemplazar el puntaje
                    max_coincidencia_parcial = coincidencia_parcial
                    if (coincidencia_parcial > max_coincidencia_parcial) & (max_coincidencia_parcial >= aprobaciones[indice]):
                        # Guardar el valor correcto
                        correccion_parcial = valor_correcto_parcial
                        # Reemplazar el puntaje
                        max_coincidencia_parial = coincidencia_parcial                        
                        # Asigno el mejor valor posible encontrado
                        df_alquileres['city2']=correccion_parcial
                        encontro = True
                        
                        
# En caso de que no se cumpla con las condiciones del macheo, mantiene el valor para ciudad inicial
            

In [518]:
# Tenemos cuatro alojamientos que no tienen ciudad ingresada
df_alquileres['city'].isna().sum()

4

In [519]:
# Consideramos que la ciudad es un dato importante, y por la cantidad la accion a tomar es elimnar esos 4 registros. Pero como en este analisis no require el uso de la columna city. lo dejamos
# df_alquileres.dropna(subset=['city'],inplace= True)

In [520]:
# Asignamos los nuevos valores a city y eliminamos la columna city2
df_alquileres['city']=df_alquileres['city2']
df_alquileres = df_alquileres.drop('city2', axis=1)

In [521]:
df_alquileres.groupby(by='city',observed=True)['city'].count()

city
AIU                                       1
AJK                                       1
AWF                                       2
BARILOCHE                                 4
BAX                                       2
                                         ..
VIEDMA                                    1
VILLA LA ANGOSTURA                       23
VILLA LA ANGOSTURA, NEUQUEN ARGENTINA     1
VILLA LLAO LLAO                           1
VILLA TRAFUL                              3
Name: city, Length: 70, dtype: int64

In [522]:
# Cambiamos el tipo de datos de city
df_alquileres['city'] = df_alquileres['city'].astype('category')

+ Columna 5: latitude

In [523]:
df_alquileres['latitude'].info()

<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: latitude
Non-Null Count  Dtype  
--------------  -----  
1028 non-null   float64
dtypes: float64(1)
memory usage: 16.1 KB


In [524]:
df_alquileres['latitude'].unique()

array([-42.7563015 , -42.75577473, -42.76800685, -42.77759   ,
       -42.77054   , -42.76833   , -42.78191   , -42.76473   ,
       -42.7716337 , -42.7538    , -42.76864   , -45.81441   ,
       -42.76506   , -42.76887   , -43.25534   , -42.91027   ,
       -42.76985   , -42.767826  , -42.77496   , -45.86703   ,
       -42.56882   , -43.27444   , -43.25614   , -42.77106   ,
       -45.86496   , -42.91138   , -42.77884   , -42.9148784 ,
       -42.9132481 , -42.91526   , -42.9074621 , -42.7687445 ,
       -42.56749   , -43.09089   , -43.25905   , -42.76854884,
       -45.86038   , -42.91053   , -42.772861  , -42.65054   ,
       -42.76724   , -43.2591    , -42.77329   , -42.91809   ,
       -42.92146   , -42.76916   , -42.752541  , -42.5117    ,
       -42.77835745, -43.25566171, -42.7756997 , -42.90797   ,
       -42.22750854, -42.77086   , -42.91956   , -42.99143   ,
       -42.56918   , -42.77094   , -42.57219   , -42.56978   ,
       -43.32639   , -43.25009918, -42.77208   , -42.76

In [525]:
# No tenemos valores nulos
df_alquileres['latitude'].isna().unique()

array([False])

+ Columna 6: longitude

In [526]:
df_alquileres['longitude'].info()

<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: longitude
Non-Null Count  Dtype  
--------------  -----  
1028 non-null   float64
dtypes: float64(1)
memory usage: 16.1 KB


In [527]:
# No tenemos valores nulos
# df_alquileres['longitude'].isna().unique()
df_alquileres['longitude'].unique()

array([-65.0375173 , -65.03726515, -65.03310833, ..., -68.30871   ,
       -68.3178    , -68.27647   ])

In [528]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   category
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 non-nu

+ Columna 7: characteristics

In [529]:
# Chequeo rapido de los valores de characteristics
df_alquileres['characteristics'].unique()

array(["['2 camas', '1 dormitorio', '1 baño']",
       "['2 camas', '1 dormitorio', '1 baño', '1 cama doble', '1 sofá cama']",
       "['3 camas', '1 dormitorio', '1 baño', 'Calificado con 5 de 5 en limpieza']",
       "['1 cama', '1 dormitorio', '1 baño', '1 cama queen', 'Calificado con 4,8 de 5 en limpieza']",
       "['2 camas', 'Monoambiente', '1 baño']",
       "['4 camas', '2 dormitorios', '2,5 baños', '1 cama king', '3 camas de una plaza', 'Calificado con 4,9 de 5 en limpieza']",
       "['3 camas', '2 dormitorios', '1 baño', 'Calificado con 4,6 de 5 en limpieza']",
       "['2 camas', 'Monoambiente', '1 baño', 'Calificado con 4,8 de 5 en limpieza']",
       "['3 camas', '2 dormitorios', '3 baños']",
       "['3 camas', '1 dormitorio', '1 baño', '2 camas queen', '1 sofá cama', 'Calificado con 5 de 5 en limpieza']",
       "['2 camas', '1 dormitorio', '1 baño', '1 cama doble', '1 sofá cama', 'Calificado con 4,9 de 5 en limpieza']",
       "['2 camas', '1 dormitorio', '1 ,5 baños'

In [530]:
# Creamos un dataframe auxiliar para trabajar con las caracteristicas
df_caracteristicas = df_alquileres[['id_alojamiento','characteristics']]
df_caracteristicas.sample(10)

,id_alojamiento,characteristics
293,742686726143047393,"['2 camas', 'Monoambiente', '1 baño', 'Estacio..."
336,902938534870515516,"['1 cama', '1 dormitorio', '1 baño', '1 cama k..."
437,48145206,"['No hay evaluaciones', '2 camas', '1 dormitor..."
731,46338322,"['3 camas', '2 dormitorios', '1 baño', '1 cama..."
270,42980235,"['1 cama', '1 dormitorio', '1 baño', '1 cama d..."
263,31021085,"['1 cama', '1 dormitorio', '1 baño', '1 cama d..."
929,51521764,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '..."
546,46324472,"['2 baños', '2 camas dobles', '4 camas de una ..."
537,53104799,"['No hay evaluaciones', '4 camas', '2 dormitor..."
320,822370130203674319,"['3 camas', '2 dormitorios', '1 ,5 baños', '1 ..."


In [531]:
# Generamos un dataframe auxiliar para poder determinar las diferentes caracteristicas posibles
df_temp = df_caracteristicas['characteristics'].str.split(
    pat=',', expand=True).add_prefix('Columna')
# Juntamos los dos dataframes auxiliares
df_caracteristicas = df_caracteristicas.merge(
    df_temp, how='left', left_index=True, right_index=True, suffixes=('', '_y'))

In [532]:
# Agragamos columnas correspondientes para asignar los valores de caracteristicas a utilizar posteriormente, seteandolas a na
df_caracteristicas['cama']=None
df_caracteristicas['dormitorio']=None
df_caracteristicas['baño']=None

# df_caracteristicas['cama']=df_caracteristicas['cama'].astype('Int64')
# df_caracteristicas['dormitorio']=df_caracteristicas['dormitorio'].astype('Int64')
# df_caracteristicas['baño']=df_caracteristicas['baño'].astype('Int64')

In [533]:
# Como vimos que existen "Monoambiente", para esos alojamientos, ponemos dormitorio en 0
df_caracteristicas['dormitorio']= np.where(df_caracteristicas['characteristics'].str.lower().str.contains('monoambiente'),0,None)

In [534]:
# EN ESTA SECCION TRATAMOS DE DETERMINAR LA CANTIDAD DE CAMAS, DORMITORIOS Y BAÑOS POR ALOJAMIENTO

# Columnas a rellenar y texto a buscar
busqueda = ['cama', 'dormitorio', 'baño']
# Recorremos por la lista de busqueda
for bus in busqueda:
    # Iteramos en el dataframe de caracteristicas
    for indice, fila in df_caracteristicas.iterrows():
        # Variable para concatenar  y formar los nombres de columnas
        ii = 2
        # Loop para encontrar el posible valor para las camas, dormitorios y baños si no los tienen, dentro de las columnas que se generaron al hacer split
        while (pd.isnull(df_caracteristicas.loc[indice,bus]) and (ii <= 14)):
            # Nombre de columna con la cual trabajar
            cc = 'Columna'+str(ii-2)
            # Id de alojamiento de la fila
            idd = fila['id_alojamiento']
            # Valor de la columna, sobre el cual realizar la busqueda
            val = fila[cc]
            # si todavia no se ha encontrado un valor acorde
            if(not pd.isnull(val)):
                # Si existe un patron que corresponda al buscado: Nro y cama o dormitorio o baño
                if bool(re.search(r'\d '+bus,val)):
                    # Obtenemos el valor que corresponde a la cantidad
                    dato = re.search(r'\d',val)                    
                    # print(fila['id_alojamiento'], val,dato.group(0))
                    # Asignamos el valor hallado en el dataframe de caracteristicas
                    df_caracteristicas.loc[indice,bus]=dato.group(0)
            # incrementamos la variabla para que salga, cuando terminen las columnas a buscar, aunque no lo encuentre
            ii+=1
            

In [535]:
df_caracteristicas.sample(20)


,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño
34,53493881,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",['2 camas','1 dormitorio','1,5 baños','1 cama doble','1 sofá cama','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,2,1,5
771,35305521,"['3 camas', '2 dormitorios', '1 baño', 'Estaci...",['3 camas','2 dormitorios','1 baño','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,3,2,1
461,53825088,"['No hay evaluaciones', '1 cama', 'Monoambient...",['No hay evaluaciones','1 cama','Monoambiente','1 baño','1 cama doble','No hay evaluaciones','No hay evaluaciones'],None,None,None,None,None,None,1,0,1
418,628315438594207176,"['4 camas', '2 dormitorios', '1 baño', 'Estaci...",['4 camas','2 dormitorios','1 baño','Estacionamiento gratis','Wifi'],None,None,None,None,None,None,None,None,4,2,1
826,755730964467686523,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",['1 cama','1 dormitorio','1 baño','Wifi','Lavarropas','Cocina','Calefacción','Calificado con 4,9 de 5 en limpieza'],None,None,None,None,1,1,1
653,48254202,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",['1 cama','Monoambiente','1 baño','1 cama queen','Estacionamiento gratis','Wifi','Calificado con 4,9 de 5 en limpieza'],None,None,None,None,None,1,0,1
165,54240811,"['1 cama', '1 dormitorio', '1 baño privado', '...",['1 cama','1 dormitorio','1 baño privado','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,1,1,1
703,667947519113196961,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",['3 camas','1 dormitorio','1 baño','1 cama queen','2 sofás cama','Estacionamiento gratis','Wifi','Calificado con 4,8 de 5 en limpieza'],None,None,None,None,3,1,1
363,842936153967499941,"['No hay evaluaciones', '1 cama', '1 dormitori...",['No hay evaluaciones','1 cama','1 dormitorio','1 baño','Wifi','No hay evaluaciones','No hay evaluaciones'],None,None,None,None,None,None,1,1,1
959,51983057,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",['1 cama','1 dormitorio','1 baño','Wifi','Lavarropas','Cocina','Calefacción','Calificado con 4,7 de 5 en limpieza'],None,None,None,None,1,1,1


In [536]:
# Revision de na faltantes para la columna baño
df_caracteristicas['baño'].info()


<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: baño
Non-Null Count  Dtype 
--------------  ----- 
1000 non-null   object
dtypes: object(1)
memory usage: 48.4+ KB


In [537]:
mask_baño = df_caracteristicas['baño'].isna()
mask_dormitorio = df_caracteristicas['dormitorio'].isna()
mask_cama = df_caracteristicas['cama'].isna()
df_caracteristicas[(mask_baño & mask_dormitorio & mask_cama)]
df_alquileres.loc[(df_alquileres['id_alojamiento'] == 13998536) | (df_alquileres['id_alojamiento'] == 20234351)]

# Para estos casos en particular, consideramos que puedes ser una especie de lugar solamente, 
# donde NO tengan baños privados, que no tengan camas porque tengan que llevar bolsa para dormir, etc 
# Y ademas no se considera que tengan dormitorio

df_caracteristicas['dormitorio']= np.where((mask_baño & mask_dormitorio & mask_cama),0,df_caracteristicas['dormitorio'])
df_caracteristicas['cama']= np.where((mask_baño & mask_dormitorio & mask_cama),0,df_caracteristicas['cama'])
df_caracteristicas['baño']= np.where((mask_baño & mask_dormitorio & mask_cama),0,df_caracteristicas['baño'])


In [538]:
# Observamos que no hay mas filas donde baño sea na que tenga datos faltantes en cama o dormitorio
mask_baño = df_caracteristicas['baño'].isna()
mask_dormitorio = df_caracteristicas['dormitorio'].isna()
mask_cama = df_caracteristicas['cama'].isna()

df_caracteristicas.loc[((mask_baño & mask_dormitorio) | (mask_baño & mask_cama)),['id_alojamiento','cama','baño','dormitorio']]

,id_alojamiento,cama,baño,dormitorio


In [539]:
# Asignacion de al menos un baño
df_caracteristicas.fillna({'baño':1},inplace= True)
mask_baño = df_caracteristicas['baño'].isna()
df_caracteristicas.loc[(mask_baño ),['id_alojamiento','cama','baño','dormitorio']]

,id_alojamiento,cama,baño,dormitorio


* Revision para la columna cama

In [540]:
# Revision de na faltantes para la columna baño
df_caracteristicas['cama'].info()


<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: cama
Non-Null Count  Dtype 
--------------  ----- 
1021 non-null   object
dtypes: object(1)
memory usage: 48.4+ KB


In [541]:
mask_cama = df_caracteristicas['cama'].isna()
df_caracteristicas.loc[(mask_cama)]

,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño
419,52371078,"['No hay evaluaciones', '1 baño privado', 'Est...",['No hay evaluaciones','1 baño privado','Estacionamiento gratis','No hay evaluaciones','No hay evaluaciones'],None,None,None,None,None,None,None,None,None,None,1
553,847798869465032246,"['1 baño', 'Estacionamiento gratis', 'Wifi']",['1 baño','Estacionamiento gratis','Wifi'],None,None,None,None,None,None,None,None,None,None,None,None,1
570,46470101,"['1 baño', '1 sofá cama', 'Estacionamiento gra...",['1 baño','1 sofá cama','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,1
575,54396194,"['1 baño', 'Estacionamiento gratis', 'Wifi', '...",['1 baño','Estacionamiento gratis','Wifi','Calificado con 4,7 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,1
577,53714616,"['1 baño', 'Estacionamiento gratis', 'Wifi', '...",['1 baño','Estacionamiento gratis','Wifi','Calificado con 4,6 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,1
579,788419802667082276,"['1 baño', 'Wifi', 'Calificado con 4,8 de 5 en...",['1 baño','Wifi','Calificado con 4,8 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,None,1
738,795027372832075610,"['1 ,5 baños', 'Estacionamiento gratis', 'Wifi...",['1,5 baños','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,5


In [542]:
# Asumimos que estos registros, tienen al menos una cama (aunque sea sofacama)
df_caracteristicas.fillna({'cama':1},inplace= True)
mask_cama = df_caracteristicas['cama'].isna()
df_caracteristicas.loc[(mask_cama)]

,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño


* Revision de la columna dormitorio

In [543]:
df_caracteristicas['dormitorio'].info()

<class 'pandas.core.series.Series'>
Index: 1028 entries, 0 to 1079
Series name: dormitorio
Non-Null Count  Dtype 
--------------  ----- 
1002 non-null   object
dtypes: object(1)
memory usage: 48.4+ KB


In [544]:
mask_dormitorio = df_caracteristicas['dormitorio'].isna()
df_caracteristicas.loc[(mask_dormitorio)]

,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño
122,782580256541651315,"['1 ,5 baños', '1 cama doble', '2 camas de una...",['1,5 baños','1 cama doble','2 camas de una plaza','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,None,1,None,5
172,20902487,"['1 baño compartido', '1 cama doble', 'Calific...",['1 baño compartido','1 cama doble','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,None,None,None,1,None,1
292,730363393464119980,"['1 baño compartido', '1 cama doble', 'Wifi', ...",['1 baño compartido','1 cama doble','Wifi','Calificado con 4,6 de 5 en limpieza'],None,None,None,None,None,None,None,None,1,None,1
335,21681812,"['1 baño compartido', '2 camas de una plaza', ...",['1 baño compartido','2 camas de una plaza','Estacionamiento gratis','Wifi','Calificado con 4,9 de 5 en limpieza'],None,None,None,None,None,None,None,2,None,1
402,41321385,"['2 baños compartidos', '1 cama doble', 'Estac...",['2 baños compartidos','1 cama doble','Estacionamiento gratis','Wifi','Calificado con 4 de 5 en limpieza'],None,None,None,None,None,None,None,None,1,None,2
419,52371078,"['No hay evaluaciones', '1 baño privado', 'Est...",['No hay evaluaciones','1 baño privado','Estacionamiento gratis','No hay evaluaciones','No hay evaluaciones'],None,None,None,None,None,None,None,None,1,None,1
426,746447501150411586,"['1,5 baños compartidos', '1 cama king', '2 ca...",['1,5 baños compartidos','1 cama king','2 camas de una plaza','Estacionamiento gratis','Wifi','Calificado con 4,7 de 5 en limpieza'],None,None,None,None,None,1,None,5
546,46324472,"['2 baños', '2 camas dobles', '4 camas de una ...",['2 baños','2 camas dobles','4 camas de una plaza','Estacionamiento gratis','Wifi','Calificado con 5 de 5 en limpieza'],None,None,None,None,None,None,None,2,None,2
553,847798869465032246,"['1 baño', 'Estacionamiento gratis', 'Wifi']",['1 baño','Estacionamiento gratis','Wifi'],None,None,None,None,None,None,None,None,None,None,1,None,1
568,53288809,"['1 baño', '1 cama doble', 'Wifi', 'Calificado...",['1 baño','1 cama doble','Wifi','Calificado con 4,7 de 5 en limpieza'],None,None,None,None,None,None,None,None,1,None,1


In [545]:
# Observamos que todos tienen alguna cama en las caracteristicas y asumimos, al menos 1 dormitorio en el alojamiento
df_caracteristicas.fillna({'dormitorio':1},inplace= True)
mask_dormitorio = df_caracteristicas['dormitorio'].isna()
df_caracteristicas.loc[(mask_dormitorio)]

,id_alojamiento,characteristics,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,cama,dormitorio,baño


In [546]:
# Verificacion general, para asegurarnos que se asignaron todos los valores
mask_baño = df_caracteristicas['baño'].isna()
mask_dormitorio = df_caracteristicas['dormitorio'].isna()
mask_cama = df_caracteristicas['cama'].isna()
df_caracteristicas.loc[(mask_baño | mask_dormitorio | mask_cama),['id_alojamiento','cama','baño','dormitorio']]

,id_alojamiento,cama,baño,dormitorio


In [547]:
df_aux= df_caracteristicas[['id_alojamiento','cama','dormitorio','baño']]
df_alquileres = df_alquileres.merge(
    df_aux,
    left_index=False,
    right_index=False,
    left_on='id_alojamiento',
    right_on='id_alojamiento',
    how='left',
    suffixes=('', '_y')        
)
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,...,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones,best_reviews,cama,dormitorio,baño
0,855050336912355203,North Beach,entire_home,4.846171,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,...,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False,2,1,1
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,4.846171,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,...,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False,2,1,1
2,800749301724167838,Departamento Bella Vista,entire_home,5.000000,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,...,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6,True,3,1,1
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.910000,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,...,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70,False,1,1,1
4,916738743930751566,Front Beach,entire_home,4.846171,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,...,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,44,False,2,0,1


In [548]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   category
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 n

+ Columna 8: check_in

In [549]:
# Chequeo rapido de los valores de characteristics
df_alquileres['check_in'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: check_in
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [556]:
df_alquileres['check_in']= pd.to_datetime(df_alquileres['check_in'],format='mixed')
df_alquileres['check_in'].sample(10)

49    2023-07-05
626   2023-09-02
91    2023-07-01
820   2023-08-16
886   2023-07-01
114   2023-07-01
327   2023-07-01
891   2023-07-01
477   2023-10-01
537   2023-08-17
Name: check_in, dtype: datetime64[ns]

+ Columna 9: check_out

In [557]:
# Chequeo rapido de los valores de characteristics
df_alquileres['check_out'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: check_out
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [558]:
df_alquileres['check_out']= pd.to_datetime(df_alquileres['check_out'],format='mixed')
df_alquileres['check_out'].sample(10)

791   2023-09-05
119   2023-07-06
411   2023-07-06
824   2023-07-06
208   2023-07-06
455   2023-07-06
767   2023-07-06
855   2023-07-06
719   2023-08-17
329   2023-07-06
Name: check_out, dtype: datetime64[ns]

In [559]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_alojamiento    1028 non-null   int64         
 1   name              1028 non-null   object        
 2   category          1028 non-null   category      
 3   rating            1028 non-null   float64       
 4   city              1024 non-null   category      
 5   latitude          1028 non-null   float64       
 6   longitude         1028 non-null   float64       
 7   characteristics   1028 non-null   object        
 8   check_in          1028 non-null   datetime64[ns]
 9   check_out         1028 non-null   datetime64[ns]
 10  price_discounted  191 non-null    object        
 11  price_original    1028 non-null   object        
 12  price_qualifier   1028 non-null   object        
 13  cant_resultados   1028 non-null   int64         
 14  fecha_scrapping   1028 n

+ Columna 10: price_original

In [589]:
# Chequeo rapido de los valores de la colkumna
df_alquileres['price_original'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: price_original
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [590]:
df_alquileres['price_original'].sample(15)

883     $85 USD
96      $38 USD
911     $37 USD
974     $69 USD
923     $26 USD
85      $42 USD
930     $81 USD
281     $31 USD
837     $55 USD
997     $35 USD
126    $130 USD
689     $85 USD
458     $99 USD
589    $150 USD
933     $90 USD
Name: price_original, dtype: object

In [591]:
# Obtengo solo los valores para el precio original
col = df_alquileres['price_original'].str.extract(pat=r'(\d+)')
df_alquileres['price_original']=col

In [593]:
df_alquileres['price_original']=df_alquileres['price_original'].astype('Int64')

In [594]:
df_alquileres['price_original'].unique()

<IntegerArray>
[ 38,  24,  43,  41,  35, 122,  68,  25, 113,  87,
 ...
  92, 228,  76, 245, 152, 111, 225, 178, 166, 480]
Length: 160, dtype: Int64

+ Columna 11: price_discounted

In [560]:
# Chequeo rapido de los valores de characteristics
df_alquileres['price_discounted'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: price_discounted
Non-Null Count  Dtype 
--------------  ----- 
191 non-null    object
dtypes: object(1)
memory usage: 8.2+ KB


In [562]:
df_alquileres['price_discounted'].sample(20)

40         NaN
609        NaN
74         NaN
454        NaN
861        NaN
571        NaN
758        NaN
58         NaN
792        NaN
293        NaN
654        NaN
173    $24 USD
786        NaN
73         NaN
338        NaN
992        NaN
812    $22 USD
746    $64 USD
682        NaN
339        NaN
Name: price_discounted, dtype: object

In [563]:
# Creo la columna, que indica si el precio ya tiene descuento
df_alquileres['discounted']=np.where(df_alquileres['price_discounted'].isna(),False,True)

In [566]:
df_alquileres[['price_discounted','discounted']].sample(20)

,price_discounted,discounted
435,NaN,False
935,NaN,False
429,NaN,False
32,NaN,False
509,NaN,False
543,NaN,False
315,NaN,False
779,NaN,False
225,NaN,False
393,NaN,False


In [586]:
# df_alquileres['price_discounted1']=df_alquileres['price_discounted'].str.extract(pat='(\d+)')
col = df_alquileres['price_discounted'].str.extract(pat=r'(\d+)')
df_alquileres['price_discounted1']=col


In [597]:
df_alquileres[['price_original','price_discounted','discounted','price_discounted1']].sample(20)

,price_original,price_discounted,discounted,price_discounted1
930,81,$73 USD,True,73
748,60,$48 USD,True,48
652,170,$139 USD,True,139
506,64,NaN,False,NaN
560,126,NaN,False,NaN
376,71,$57 USD,True,57
481,99,NaN,False,NaN
563,79,NaN,False,NaN
479,25,NaN,False,NaN
243,218,NaN,False,NaN


In [598]:
df_alquileres['price_discounted1']=np.where( df_alquileres['price_discounted1'].isna(),df_alquileres['price_original'],df_alquileres['price_discounted1'])

In [600]:
df_alquileres['price_discounted'] = df_alquileres['price_discounted1']

In [601]:
df_alquileres[['price_original','price_discounted','discounted','price_discounted1']].sample(20)

,price_original,price_discounted,discounted,price_discounted1
321,34,34,False,34
575,60,60,False,60
505,69,69,False,69
762,84,84,False,84
372,79,79,False,79
593,90,80,True,80
904,59,59,False,59
616,63,63,False,63
827,54,54,False,54
874,77,77,False,77


In [602]:
df_alquileres = df_alquileres.drop('price_discounted1', axis=1)

In [611]:
df_alquileres[['price_original','price_discounted','discounted']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   price_original    1028 non-null   Int64
 1   price_discounted  1028 non-null   Int64
 2   discounted        1028 non-null   bool 
dtypes: Int64(2), bool(1)
memory usage: 19.2 KB


In [610]:
df_alquileres['price_discounted'].unique()

<IntegerArray>
[ 38,  18,  43,  41,  35, 122,  40,  25,  45,  87,
 ...
  92,  97, 203,  76, 245, 134, 225, 178, 133, 480]
Length: 158, dtype: Int64

In [609]:
df_alquileres['price_discounted']=(pd.to_numeric(df_alquileres['price_discounted'], errors='coerce')).astype('Int64')

In [612]:
df_alquileres[['price_original','price_discounted','discounted']].sample(20)

,price_original,price_discounted,discounted
661,400,356,True
997,35,35,False
7,25,25,False
994,50,50,False
383,60,60,False
824,68,59,True
381,32,32,False
530,250,250,False
868,24,24,False
847,50,50,False


+ Columna 12: price_qualifier

In [613]:
df_alquileres['price_qualifier'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: price_qualifier
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [614]:
df_alquileres['price_qualifier'].unique()

array(['noche'], dtype=object)

In [615]:
df_alquileres['price_qualifier']= df_alquileres['price_qualifier'].astype('category')

+ Columna 13: cant_resultados

In [616]:
df_alquileres['cant_resultados'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: cant_resultados
Non-Null Count  Dtype
--------------  -----
1028 non-null   int64
dtypes: int64(1)
memory usage: 8.2 KB


In [617]:
df_alquileres['cant_resultados'].unique()

array([1000,  298], dtype=int64)

In [618]:
df_alquileres['cant_resultados'].sample(20)

991    1000
639    1000
978    1000
127    1000
755    1000
661    1000
726    1000
268     298
200    1000
569    1000
792    1000
11     1000
359     298
453     298
828    1000
610    1000
387     298
631    1000
846    1000
923    1000
Name: cant_resultados, dtype: int64

+ Columna 14: fecha_scrapping

In [619]:
df_alquileres['fecha_scrapping'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: fecha_scrapping
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [620]:
df_alquileres['fecha_scrapping'].sample()

655    2023-06-24
Name: fecha_scrapping, dtype: object

In [621]:
df_alquileres['fecha_scrapping']=pd.to_datetime(df_alquileres['fecha_scrapping'],format='mixed')

+ Columna 15: fecha_cambio_bna

In [622]:
df_alquileres['fecha_cambio_bna'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: fecha_cambio_bna
Non-Null Count  Dtype 
--------------  ----- 
1028 non-null   object
dtypes: object(1)
memory usage: 8.2+ KB


In [623]:
df_alquileres['fecha_cambio_bna'].sample()

762    2023-05-01
Name: fecha_cambio_bna, dtype: object

In [624]:
df_alquileres['fecha_cambio_bna'].unique()

array(['2023-05-01'], dtype=object)

In [625]:
df_alquileres['fecha_cambio_bna']= pd.to_datetime(df_alquileres['fecha_cambio_bna'],format='mixed')

+ Columna 16: cambio_bna

In [626]:
df_alquileres['cambio_bna'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1028 entries, 0 to 1027
Series name: cambio_bna
Non-Null Count  Dtype  
--------------  -----  
1028 non-null   float64
dtypes: float64(1)
memory usage: 8.2 KB


In [627]:
df_alquileres['cambio_bna'].sample(20)

59     239.919355
171    239.919355
558    239.919355
286    239.919355
689    239.919355
605    239.919355
261    239.919355
925    239.919355
757    239.919355
861    239.919355
817    239.919355
800    239.919355
756    239.919355
543    239.919355
628    239.919355
646    239.919355
131    239.919355
734    239.919355
136    239.919355
971    239.919355
Name: cambio_bna, dtype: float64

In [628]:
df_alquileres['cambio_bna'].unique()

array([239.91935484])

In [629]:
df_alquileres['cambio_bna']=df_alquileres['cambio_bna'].astype('float')

+ Columna 17: provincia

In [631]:
df_alquileres['provincia']=df_alquileres['provincia'].astype('category')

In [634]:
# Ajustamos tipos de datos de las columnas cama, dormitorio, baño
df_alquileres['cama']=(pd.to_numeric(df_alquileres['cama'], errors='coerce')).astype('Int64')
df_alquileres['baño']=(pd.to_numeric(df_alquileres['baño'], errors='coerce')).astype('Int64')
df_alquileres['dormitorio']=(pd.to_numeric(df_alquileres['dormitorio'], errors='coerce')).astype('Int64')

In [635]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_alojamiento    1028 non-null   int64         
 1   name              1028 non-null   object        
 2   category          1028 non-null   category      
 3   rating            1028 non-null   float64       
 4   city              1024 non-null   category      
 5   latitude          1028 non-null   float64       
 6   longitude         1028 non-null   float64       
 7   characteristics   1028 non-null   object        
 8   check_in          1028 non-null   datetime64[ns]
 9   check_out         1028 non-null   datetime64[ns]
 10  price_discounted  1028 non-null   Int64         
 11  price_original    1028 non-null   Int64         
 12  price_qualifier   1028 non-null   category      
 13  cant_resultados   1028 non-null   int64         
 14  fecha_scrapping   1028 n